This is a script for geocoding and plotting spatial information from an EXCEL spreadsheet with an "Addresses" column. The API used is **GeoNames.**

GeoNames is mainly using REST APIs. It offers 40 different webservices.

**Geocoder** for Python supports the following ones:

*   (geocoding) retrieve GeoNames’s geocoded data from a query string, and 
various filters
*   (details) retrieve all geonames data for a given geonames_id
*   (children) retrieve the hierarchy of a given geonames_id
*   (hierarchy) retrieve all children for a given geonames_id

Full documentation: https://geocoder.readthedocs.io/providers/GeoNames.html

The first step is to get COLAB working:


In [1]:
## mount drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


A file path needs to be defined for storing input or output files linked with this script:

In [2]:
directory="/content/drive/My Drive/Colab_DigiKAR/"

Now we can install packages that are not part of Python's standard distribution but are necessary for geocoding and plotting maps. There will most likely be a dependency error for NumPy, but the script should still work.

In [3]:
## install packages that are not part of Python's standard distribution

!pip install geocoder
!pip install basemap
!pip install ipyleaflet
!pip install geojson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 3.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 83.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency confl

Now that all packages are installed, we can read the input data (in this case from Github or Google Drive) and display the content in a table.

In [4]:
## import relevant packages for geocoding as well as reading and writing data
import pandas as pd
import geocoder
# command needed for correct plotting in Jupyter Notebooks:
%matplotlib inline 
import pandas as pd
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import os
import json
from geojson import Feature, FeatureCollection, Point

## geocode data from spreadsheet

# input addresses in EXCEL format and read
infile="https://github.com/MonikaBarget/GeoHumTutorials/blob/master/Colab_Geocoding/Addresses_AP3.xlsx?raw=true"
#infile=directory+"xxxxx.xlsx" # alternative input from Google Drive
addresses_df = pd.read_excel(infile)
display(addresses_df)

,Addresses
0,Aachen
1,Abderode
2,Alach
3,Algesheim
4,Altdorf
...,...
169,Wien
170,Worbis
171,Worms
172,Würzburg


Now we will use the Pandas package to read the content of the address column to a so-called DataFrame. A DataFrame is a data structure that organizes data into a 2-dimensional table of rows and columns, much like a spreadsheet. This 2-dimensional structure is often used to manipulate data with programming languages. Our "manipulation" is the act of geocoding.

In [5]:
# read information from address column to dataframe
addresses = addresses_df["Addresses"].values.tolist()

latitudes = []
longitudes = []
ids = []
g_addresses = []

# geocode each address in file
for address in addresses:
	try:
		g = geocoder.geonames(address, key="YOURKEY", featureClass='A')

		if g and len(g):
			geonames_address = g.address
			longitude = g.lng
			latitude = g.lat
			id = g.geonames_id
			#print(geonames_address, longitude, latitude, id) # OPTIONAL: print individual output
		else:
			geonames_address = "0"
			longitude = "0"
			latitude = "0"
			id = "0"

	# add information to lists
		g_addresses.append(geonames_address)
		latitudes.append(latitude)
		longitudes.append(longitude)
		ids.append(id)

	except ValueError:
		print("No more data in file.")

# write information to new columns in dataframe
addresses_df["latitudes"] = latitudes
addresses_df["longitudes"] = longitudes
addresses_df["ids"] = ids
addresses_df["geonames address"] = g_addresses

print("All addresses geocoded!")


All addresses geocoded!


If all addresses have been successfully geocoded, the next step is to check the geocoding and write the results to a new EXCEL file. 


In [7]:
# view geocoded data
display(addresses_df)

# write geocoded places to new file
addresses_df.to_excel(directory+"Addresses_AP3_Geocoded_withID.xlsx")

,Addresses,latitudes,longitudes,ids,geonames address
0,Aachen,50.75389,6.24194,3247448,Städteregion Aachen
1,Abderode,0,0,0,0
2,Alach,0,0,0,0
3,Algesheim,49.94992,8.00946,6557928,Gau-Algesheim
4,Altdorf,46.88834,8.64091,7285057,Altdorf (UR)
...,...,...,...,...,...
169,Wien,48.2082,16.37169,2761367,Vienna
170,Worbis,51.37915,10.33333,7602636,Leinefelde-Worbis
171,Worms,49.66306,8.33583,3247467,Kreisfreie Stadt Worms
172,Würzburg,49.7822,9.94041,3220897,Kreisfreie Stadt Würzburg


Our geocoded data have been written to a new EXCEL file, which is handy for further (manual) data cleaning and data enrichment. But EXCEL is unfortunately not a file format which GIS applications can handle. This is why we also need to export our geocoded data to GeoJSON.

The conversion of a DataFrame to GeoJSON follows the instructions in the following tutorial by Geoff Boeing:

https://notebook.community/captainsafia/nteract/applications/desktop/example-notebooks/pandas-to-geojson

In [8]:
# convert coordinates to floats

addresses_df['latitudes'] = addresses_df['latitudes'].astype(float)
addresses_df['longitudes'] = addresses_df['longitudes'].astype(float)

# ignore places that have not been geocoded

df_geo = addresses_df.dropna(subset=['latitudes', 'longitudes'], axis=0, inplace=False)

# combine information in GeoJSON fromat

def df_to_geojson(df, properties, lat='latitudes', lon='longitudes'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

cols = ['Addresses', 'ids', 'geonames address']
geojson = df_to_geojson(df_geo, cols)

with open(directory+'AP3.geojson', 'w', encoding='utf-8') as f:
    json.dump(geojson, f, ensure_ascii=False)

Your Google Drive should now contain a file with the "geojson" file ending. We can check if this file has been created and if it is well-formed.

In [9]:
## double-check if GeoJSON file has been created and is well-formed

# load GeoJSON data

with open(directory+'AP3.geojson', 'r') as f2:
    data = json.load(f2)
    print(data)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'Addresses': 'Aachen', 'ids': 3247448, 'geonames address': 'Städteregion Aachen'}, 'geometry': {'type': 'Point', 'coordinates': [6.24194, 50.75389]}}, {'type': 'Feature', 'properties': {'Addresses': 'Abderode', 'ids': '0', 'geonames address': '0'}, 'geometry': {'type': 'Point', 'coordinates': [0.0, 0.0]}}, {'type': 'Feature', 'properties': {'Addresses': 'Alach', 'ids': '0', 'geonames address': '0'}, 'geometry': {'type': 'Point', 'coordinates': [0.0, 0.0]}}, {'type': 'Feature', 'properties': {'Addresses': 'Algesheim', 'ids': 6557928, 'geonames address': 'Gau-Algesheim'}, 'geometry': {'type': 'Point', 'coordinates': [8.00946, 49.94992]}}, {'type': 'Feature', 'properties': {'Addresses': 'Altdorf', 'ids': 7285057, 'geonames address': 'Altdorf (UR)'}, 'geometry': {'type': 'Point', 'coordinates': [8.64091, 46.88834]}}, {'type': 'Feature', 'properties': {'Addresses': 'Alzey', 'ids': 2956708, 'geonames address': 'Lan

Now we can plot the geocoded data to an interactive map. The code below is partly based on an Ipyleaflet Tutorial provided by the *Carpentries Incubator*:

https://carpentries-incubator.github.io/jupyter_maps/01-introduction/index.html

In [10]:
## plot geocoded data on interactive map

# initialise interactive map

from ipyleaflet import Map, basemaps, GeoJSON, LayersControl
import random

# customise map

map = Map(center = (55, 7), zoom = 5, min_zoom = 1, max_zoom = 20, 
    basemap=basemaps.Stamen.Terrain)

# add functionality to add or remove layers to map itself

map.add_control(LayersControl())

def random_color(feature):
    return {
        'color': 'black',
        'fillColor': random.choice(['red', 'yellow', 'green', 'orange']),
    }

geo_json = GeoJSON(
    data=data,
    style={
        'opacity': 1, 'dashArray': '7', 'fillOpacity': 0.1, 'weight': 2
    },
    hover_style={
        'color': 'red', 'dashArray': '0', 'fillOpacity': 0.5
    },
    style_callback=random_color
)

# add geocoded data to map

map.add_layer(geo_json)

map


Map(center=[55, 7], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

Congratulations, you have just plotted a new map! At the moment, the map only has markers for the point geometries but no pop-up labels. To embed those, other Python packages will need to be imported first. I will add pop-ups in the next development step. 

Notebook created by: Monika Barget

Latest update: 26 January 2023